In [99]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.autograd import Variable
import math
import tqdm
import pdb
from scipy.sparse import csr_matrix, linalg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings(action='ignore')


In [100]:
books = pd.read_csv("../data/books.csv")
train = pd.read_csv("../data/train_ratings.csv")
test = pd.read_csv("../data/test_ratings.csv")
users = pd.read_csv("../data/users.csv")
sub = pd.read_csv("../data/sample_submission.csv")
total = pd.read_csv("../data/total_ratings.csv")

In [101]:
train

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9
...,...,...,...
306790,278843,0743525493,7
306791,278851,067161746X,6
306792,278851,0884159221,7
306793,278851,0912333022,7


In [102]:
count=train.groupby("user_id").size()

In [103]:
dfcount = pd.DataFrame(count, columns=["count"])

In [104]:
dfcount

,count
user_id,
8,7
9,1
12,1
14,2
16,1
...,...
278846,1
278849,1
278851,12


In [105]:
train.merge(dfcount,on="user_id").to_csv("train_user_count.csv")

In [106]:
newtrain = train.merge(dfcount,on="user_id")
newtrain

,user_id,isbn,rating,count
0,8,0002005018,4,7
1,8,074322678X,4,7
2,8,0887841740,2,7
3,8,1552041778,2,7
4,8,1567407781,6,7
...,...,...,...,...
306790,278411,0446608831,8,1
306791,278621,1550390961,8,1
306792,278636,0375507299,2,1
306793,278659,0345330293,10,1


In [114]:
newtrain[newtrain["count"]==1].iloc[:17308, :]

,user_id,isbn,rating,count
14,123629,0002005018,8,1
15,200273,0002005018,8,1
6781,198013,0399135782,7,1
6782,212449,0399135782,7,1
7077,252921,0399135782,8,1
...,...,...,...,...
276550,109230,0385503954,5,1
276551,118896,0385503954,9,1
276552,139892,0385503954,7,1
276553,175048,0385503954,6,1


In [115]:
newtrain[newtrain["count"]==1].iloc[17308:, :]

,user_id,isbn,rating,count
276557,231793,0385503954,1,1
276558,244215,0385503954,6,1
276559,253330,0385503954,9,1
276560,259342,0385503954,9,1
276561,50156,0385504470,7,1
...,...,...,...,...
306790,278411,0446608831,8,1
306791,278621,1550390961,8,1
306792,278636,0375507299,2,1
306793,278659,0345330293,10,1


In [108]:
newtrain[newtrain["count"]!=1]

,user_id,isbn,rating,count
0,8,0002005018,4,7
1,8,074322678X,4,7
2,8,0887841740,2,7
3,8,1552041778,2,7
4,8,1567407781,6,7
...,...,...,...,...
306759,276664,0004703723,9,3
306760,276664,0631189629,9,3
306761,276664,0747205051,7,3
306762,276729,052165615X,1,2


In [75]:
count=test.groupby("user_id").size()

In [76]:
dfcount = pd.DataFrame(count, columns=["count"])

In [77]:
dfcount

,count
user_id,
14,1
17,2
51,1
53,2
73,1
...,...
278836,1
278843,5
278844,1


In [91]:
newtest = test.merge(dfcount,on="user_id")

In [98]:
newtest[newtest["count"]==1]["user_id"]

1433     157969
1472     263256
1484      12923
1530      21264
1531      31299
          ...  
76694    277514
76695    277565
76696    277680
76697    278257
76698    278543
Name: user_id, Length: 16785, dtype: int64

In [88]:
newtrain[newtrain["count"]==1]["user_id"]

14        123629
15        200273
6781      198013
6782      212449
7077      252921
           ...  
306790    278411
306791    278621
306792    278636
306793    278659
306794    278713
Name: user_id, Length: 34617, dtype: int64

In [29]:
train.shape

(306795, 3)

In [26]:
34617/306795

0.1128343030362294

In [ ]:
34617/306795

In [3]:
train["user_id"].nunique()

59803

In [10]:
train_user_count = train.groupby('user_id').size()
train_user_count

user_id
8          7
9          1
12         1
14         2
16         1
          ..
278846     1
278849     1
278851    12
278852     1
278854     3
Length: 59803, dtype: int64

In [13]:
train_user_count.value_counts()

1      34617
2       7870
3       4093
4       2539
5       1698
       ...  
375        1
353        1
810        1
151        1
163        1
Length: 246, dtype: int64

In [14]:
test["user_id"].nunique()

26167

In [18]:
test.groupby("user_id").size()

user_id
14        1
17        2
51        1
53        2
73        1
         ..
278836    1
278843    5
278844    1
278851    2
278854    3
Length: 26167, dtype: int64